# Exercise - Neural Network

The data set for this exercise is from the banking industry. It contains data about the home loans of 2,500 bank clients. Each row represents a single loan. The columns include the characteristics of the client who used a loan. This is a binary classification task: predict whether a loan will be bad or not (1=Yes, 0=No). This is an important task for banks to prevent bad loans from being issued.

## Description of Variables

The description of variables are provided in "Loan - Data Dictionary.docx"

## Goal

Use the **loan.csv** data set and build a model to predict **BAD**. 



# Read and Prepare the Data

In [116]:
# Common imports

import pandas as pd
import numpy as np

np.random.seed(42)

# Get the data

In [117]:
#We will predict the "price" value in the data set:
loan = pd.read_csv("loan.csv")
loan.head()

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,0,25900,61064.0,94714.0,DebtCon,Office,2.0,0.0,0.0,98.809375,0.0,23.0,34.565944
1,0,26100,113266.0,182082.0,DebtCon,Sales,18.0,0.0,0.0,304.852469,1.0,31.0,33.193949
2,1,50000,220528.0,300900.0,HomeImp,Self,5.0,0.0,0.0,0.000000,0.0,2.0,NaN
3,1,22400,51470.0,68139.0,DebtCon,Mgr,9.0,0.0,0.0,31.168696,2.0,8.0,37.952180
4,0,20900,62615.0,87904.0,DebtCon,Office,5.0,NaN,NaN,177.864849,NaN,15.0,36.831076


# Split data (train/test)

In [118]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(loan, test_size=0.3)

# Data Prep

Perform your data prep here. You can use pipelines like we do in the tutorials. Otherwise, feel free to use your own data prep steps. Eventually, you should do the following at a minimum:<br>
- Separate inputs from target<br>
- Impute/remove missing values<br>
- Standardize the continuous variables<br>
- One-hot encode categorical variables<br>

In [119]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

## Separate the target variable 

In [120]:
train_target = train['BAD']
test_target = test['BAD']

train_inputs = train.drop(['BAD'], axis=1)
test_inputs = test.drop(['BAD'], axis=1)

## Checking for Missing Values

In [121]:
train.isna().sum()

BAD          0
LOAN         0
MORTDUE    151
VALUE       61
REASON      89
JOB         74
YOJ        156
DEROG      194
DELINQ     165
CLAGE      105
NINQ       153
CLNO        76
DEBTINC    556
dtype: int64

In [122]:
test.isna().sum()

BAD          0
LOAN         0
MORTDUE     71
VALUE       30
REASON      30
JOB         32
YOJ         60
DEROG       76
DELINQ      61
CLAGE       41
NINQ        59
CLNO        29
DEBTINC    262
dtype: int64

## Feature Engineering: Derive a new column

Examples:
- Ratio of delinquent to total number of credit lines
- Ratio of loan to value of current property
- Convert yr_renovated to a binary variable (i.e., renovated or not)
- (etc.)

In [123]:
#Ratio of delinquent to total number of credit lines

def new_col(df):
    
    df1 = df.copy()    
    df1['CLNO'] = np.where(df1['DELINQ'] > 0, 1, 0)
    return df1[['CLNO']]

In [124]:
new_col(train)

,CLNO
1552,0
2290,0
1398,0
1775,1
2299,0
...,...
1638,1
1095,1
1130,0
1294,0


##  Identify the numeric, binary, and categorical columns

In [125]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [126]:
numeric_columns

['LOAN',
 'MORTDUE',
 'VALUE',
 'YOJ',
 'DEROG',
 'DELINQ',
 'CLAGE',
 'NINQ',
 'CLNO',
 'DEBTINC']

In [127]:
categorical_columns

['REASON', 'JOB']

In [128]:
# define your feature engineered column(s) here
feat_eng_columns = ['DELINQ']

# Pipeline

In [129]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [130]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [131]:
# Create a pipeline for the transformed column here
my_new_column = Pipeline(steps=[('my_new_column', FunctionTransformer(new_col))])

In [132]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
       ('trans', my_new_column, feat_eng_columns)],   
        remainder='passthrough')
#passtrough is an optional step. You don't have to use it.

# Transform: fit_transform() for TRAIN

In [133]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)
train_x

array([[-0.31412013, -1.30301181, -0.86148829, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.45733454,  0.7398414 ,  0.58636192, ...,  0.        ,
         0.        ,  0.        ],
       [-1.10330939,  0.2001631 ,  0.18146318, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.21657988, -0.83000156, -0.82081329, ...,  0.        ,
         0.        ,  0.        ],
       [-0.46486414,  1.79196675,  1.36974799, ...,  0.        ,
         0.        ,  0.        ],
       [-0.31412013, -0.08740643, -0.21782887, ...,  0.        ,
         0.        ,  0.        ]])

In [134]:
train_x.shape

(1750, 21)

# Tranform: transform() for TEST

In [135]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)
test_x

array([[ 0.06717356,  0.36706438,  0.32127798, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.32432512,  0.57631513,  0.42769944, ...,  1.        ,
         0.        ,  0.        ],
       [-0.33185472,  0.41209537,  0.12227549, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.8993616 , -0.51071616, -0.32326299, ...,  0.        ,
         0.        ,  1.        ],
       [-0.5446698 , -0.82706576, -0.8366813 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.5446698 , -0.06422056, -0.11380525, ...,  0.        ,
         0.        ,  0.        ]])

In [136]:
test_x.shape

(750, 21)

# Calculate the Baseline

In [137]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(train_x, train_target)

DummyClassifier(strategy='most_frequent')

In [138]:
from sklearn.metrics import accuracy_score

In [139]:
#Baseline Train Accuracy
dummy_train_pred = dummy_clf.predict(train_x)

baseline_train_acc = accuracy_score(train_target, dummy_train_pred)

print('Baseline Train Accuracy: {}' .format(baseline_train_acc))

Baseline Train Accuracy: 0.6034285714285714


In [140]:
#Baseline Test Accuracy
dummy_test_pred = dummy_clf.predict(test_x)

baseline_test_acc = accuracy_score(test_target, dummy_test_pred)

print('Baseline Test Accuracy: {}' .format(baseline_test_acc))

Baseline Test Accuracy: 0.5773333333333334


# Train a shallow (one-layer) NN model

In [141]:
from sklearn.neural_network import MLPClassifier

#Default settings create 1 hidden layer with 100 neurons
mlp_clf = MLPClassifier(hidden_layer_sizes=(100, ))

mlp_clf.fit(train_x, train_target)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier()

In [142]:
from sklearn.metrics import accuracy_score

### Calculate the accuracy

In [143]:
#Predict the train values
train_y_pred = mlp_clf.predict(train_x)
    
#Train accuracy
accuracy_score(train_target, train_y_pred)

0.9148571428571428

In [144]:
#Predict the test values
test_y_pred = mlp_clf.predict(test_x)

#Test accuracy
accuracy_score(test_target, test_y_pred)

0.8346666666666667

# Train a deep (multi-layered) NN model 

In [145]:
dnn_clf = MLPClassifier(hidden_layer_sizes=(50,25,100),
                       max_iter=1000,
                       early_stopping=True,
                      alpha = 0.1)

dnn_clf.fit(train_x, train_target)

MLPClassifier(alpha=0.1, early_stopping=True, hidden_layer_sizes=(50, 25, 100),
              max_iter=1000)

In [146]:
#Let's check the number of iterations:
dnn_clf.n_iter_

34

In [147]:
#Let's check the number of layers:
dnn_clf.n_layers_

5

In [148]:
from sklearn.metrics import accuracy_score

### Calculate the accuracy

In [149]:
#Predict the train values
train_y_pred = dnn_clf.predict(train_x)

#Train accuracy
accuracy_score(train_target, train_y_pred)

0.8102857142857143

In [150]:
#Predict the test values
test_y_pred = dnn_clf.predict(test_x)

#Test accuracy
accuracy_score(test_target, test_y_pred)

0.7626666666666667

# Optional: try grid search

In [151]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_grid = {'max_depth': randint(low=5, high=20), 
              'min_samples_leaf': randint(low=5, high=20)}

tree_gs = RandomizedSearchCV(DecisionTreeClassifier(), param_grid, 
                             n_iter=15, cv=5, verbose=1,
                             scoring='accuracy',
                             return_train_score=True)

tree_gs.fit(train_x, train_target)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_iter=15,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000023AE35F66D0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000023AE35F6D60>},
                   return_train_score=True, scoring='accuracy', verbose=1)

In [152]:
cvres = tree_gs.cv_results_

for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

0.7977142857142858 {'max_depth': 18, 'min_samples_leaf': 5}
0.8068571428571427 {'max_depth': 16, 'min_samples_leaf': 8}
0.8177142857142858 {'max_depth': 7, 'min_samples_leaf': 12}
0.8062857142857144 {'max_depth': 16, 'min_samples_leaf': 13}
0.808 {'max_depth': 16, 'min_samples_leaf': 8}
0.8039999999999999 {'max_depth': 9, 'min_samples_leaf': 6}
0.7948571428571429 {'max_depth': 15, 'min_samples_leaf': 6}
0.8240000000000001 {'max_depth': 6, 'min_samples_leaf': 10}
0.7965714285714286 {'max_depth': 14, 'min_samples_leaf': 5}
0.8137142857142857 {'max_depth': 10, 'min_samples_leaf': 9}
0.808 {'max_depth': 17, 'min_samples_leaf': 14}
0.808 {'max_depth': 10, 'min_samples_leaf': 7}
0.7971428571428573 {'max_depth': 18, 'min_samples_leaf': 19}
0.8051428571428572 {'max_depth': 17, 'min_samples_leaf': 14}
0.8234285714285713 {'max_depth': 6, 'min_samples_leaf': 11}


In [153]:
#Find the best parameter set
tree_gs.best_params_

{'max_depth': 6, 'min_samples_leaf': 10}

In [154]:
tree_gs.best_estimator_

DecisionTreeClassifier(max_depth=6, min_samples_leaf=10)

In [155]:
#Train accuracy:
train_y_pred = tree_gs.best_estimator_.predict(train_x)

print(accuracy_score(train_target, train_y_pred))

0.8685714285714285


In [156]:
#Test accuracy:
test_y_pred = tree_gs.best_estimator_.predict(test_x)

print(accuracy_score(test_target, test_y_pred))

0.844
